In [5]:
import os
import codecs
from metal.mmtl.debugging.slicing import *

ModuleNotFoundError: No module named 'metal.mmtl.debugging.slicing'

## RTE Slices

In [3]:
func_name = 'morepeople'
task_name = 'RTE'
split = 'dev'
tsv_path = f'/dfs/scratch0/mccreery/glue_slices/{task_name}/{split}.tsv'
save_path = f'/dfs/scratch0/mccreery/glue_slices/{task_name}_{func_name}/{split}.tsv'

In [61]:
config = get_task_tsv_config(task_name,split)

sent1_idx = config["sent1_idx"]
sent2_idx = config["sent2_idx"]
filtered_rows = []
all_rows = []
with codecs.open(tsv_path, "r", "utf-8") as data_fh:
    filtered_rows.append(data_fh.readline())
    all_rows.append(data_fh.readline())
    for i, row in enumerate(data_fh):
        sent1 = row.split('\t')[sent1_idx]
        in_slice = slice_fn(sent1)
        if in_slice:
            filtered_rows.append(row)
        all_rows.append(row)
print(f'slice prop: {len(filtered_rows)}/{len(all_rows)}')

slice prop: 109/277


In [45]:
with codecs.open(save_path, "w", "utf-8") as data_fh:
    for row in filtered_rows:
        data_fh.write(row)

109

## COLA Slices 

In [77]:
split = 'train'
tsv_path = f'/dfs/scratch0/chami/glue_slices/CoLA/{split}.tsv'
save_path = f'/dfs/scratch0/chami/glue_slices/CoLA_long/{split}.tsv'

In [78]:
slice_fn = lambda sent: len(sent.split()) > 10
sent1_idx = 3
filtered_rows = []
all_rows = []
with codecs.open(tsv_path, "r", "utf-8") as data_fh:
    filtered_rows.append(data_fh.readline())
    all_rows.append(data_fh.readline())
    for i, row in enumerate(data_fh):
        sent1 = row.split('\t')[sent1_idx]
        #print(sent1, len(sent1.split()))
        in_slice = slice_fn(sent1)
        if in_slice:
            filtered_rows.append(row)
        all_rows.append(row)
print(f'slice prop: {len(filtered_rows)}/{len(all_rows)}')

slice prop: 1429/8550


In [80]:
with codecs.open(save_path, "w", "utf-8") as data_fh:
    for row in filtered_rows:
        data_fh.write(row)

## SST-2 Slices

In [ ]:
split = 'train'
tsv_path = f'/dfs/scratch0/chami/glue_slices/CoLA/{split}.tsv'
save_path = f'/dfs/scratch0/chami/glue_slices/CoLA_long/{split}.tsv'

slice_fn = lambda sent: len(sent.split()) > 10

sent1_idx = 3
filtered_rows = []
all_rows = []
with codecs.open(tsv_path, "r", "utf-8") as data_fh:
    filtered_rows.append(data_fh.readline())
    all_rows.append(data_fh.readline())
    for i, row in enumerate(data_fh):
        sent1 = row.split('\t')[sent1_idx]
        #print(sent1, len(sent1.split()))
        in_slice = slice_fn(sent1)
        if in_slice:
            filtered_rows.append(row)
        all_rows.append(row)
print(f'slice prop: {len(filtered_rows)}/{len(all_rows)}')

In [ ]:
with codecs.open(save_path, "w", "utf-8") as data_fh:
    for row in filtered_rows:
        data_fh.write(row)